# Limpieza de Datos para Clustering de series de tiempo

### Manuel Padilla garcía

In [ ]:
import os
import pandas as pd
import numpy as np
import glob
import itertools
import matplotlib.pyplot as plt
import random

from datetime import datetime, timedelta
from time import time
from tqdm import tqdm

from tslearn.clustering import KernelKMeans
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.utils import to_time_series_dataset

from tslearn.metrics import cdist_dtw
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score


folder = "data"

In [ ]:
# Ruta del archivo Parquet
file = "data/BTC-USDT.parquet"

# Leer todo el archivo Parquet
data = pd.read_parquet(file)

# Seleccionar las últimas 10 filas
rows = data.tail(10)

# Configurar pandas para que no use la notación científica
pd.set_option('display.float_format', '{:.4f}'.format)

# Mostrar las últimas 10 filas
print(rows)


### 1. ELIMINAR ARCHIVOS NO -USDT

In [ ]:
def removeNoUSDTFiles(folder):
    # Obtener la lista de archivos .parquet en la carpeta
    parquetFiles = glob.glob(os.path.join(folder, "*.parquet"))

    for file in parquetFiles:
        # Obtener el nombre del archivo sin la extensión
        fileName = os.path.splitext(os.path.basename(file))[0]
        # Verificar si el archivo no termina en "-USDT"
        if not fileName.endswith("-USDT"):
            # Eliminar el archivo
            os.remove(file)
            print(f"Se eliminó {file}")

removeNoUSDTFiles(folder)

### 2. ELIMINAR ARCHIVOS DE CRIPTOS UP Y DOWN

In [ ]:
def removeUpDownFiles(folder, prefixes):
    # Listar todos los archivos en la carpeta
    files = os.listdir(folder)

    for file in files:
        # Verificar si el archivo tiene alguno de los prefijos especificados
        for prefix in prefixes:
            if prefix in file:
                # Eliminar el archivo
                filePath = os.path.join(folder, file)
                os.remove(filePath)
                print(f"Se eliminó {file}")


# Prefijos de los archivos a eliminar
removePrefixes = ["UP-", "DOWN-"]

# Llamar a la función para eliminar los archivos
removeUpDownFiles(folder, removePrefixes)


### 3. ELIMINAR COLUMNAS

In [ ]:
# Columnas a eliminar
fields = ['high','low','close','volume','number_of_trades',
            'taker_buy_base_asset_volume','taker_buy_quote_asset_volume']

def removeFields(folder, fields):
    # Listar todos los archivos Parquet en la carpeta
    files = [f for f in os.listdir(folder) if f.endswith('.parquet')]

    for file in files:
        # Ruta completa
        filePath = os.path.join(folder, file)

        # Leer el archivo Parquet
        df = pd.read_parquet(filePath)

        # Eliminar las columnas especificadas
        df = df.drop(columns=fields, errors='ignore')
        
        # Guardar el nuevo DataFrame sin las columnas en un nuevo archivo Parquet
        df.to_parquet(filePath)


# Procesar archivos Parquet
removeFields(folder, fields)


### 4. QUITAR ÍNDICES DE ARCHIVOS PARQUET

In [ ]:
# Listar todos los archivos Parquet en la carpeta
files = [f for f in os.listdir(folder) if f.endswith('.parquet')]

for file in files:
    # Ruta completa del archivo Parquet
    filePath = os.path.join(folder, file)

    # Leer el archivo Parquet
    df = pd.read_parquet(filePath)

    # Resetear el índice para convertir el campo de fecha en una columna
    df = df.reset_index()

    # Guardar el DataFrame actualizado en el archivo Parquet
    df.to_parquet(filePath, index=False)

### 5. ELIMINAR FECHAS

In [ ]:
def dateFilter(file):
    # Leer el archivo Parquet
    df = pd.read_parquet(file)

    # Reemplaza 'open_time' con el nombre correcto de la columna que contiene la fecha y hora de apertura

    # Convertir la columna 'open_time' a tipo datetime
    df['open_time'] = pd.to_datetime(df['open_time'])

    # Filtrar las filas por fecha
    df_DateFiltered = df[(df['open_time'] >= '2020-01-01') & (df['open_time'] < '2023-01-01')]

    return df_DateFiltered

def removeDates(folder):
    # Listar todos los archivos Parquet en la carpeta
    files = [f for f in os.listdir(folder) if f.endswith('.parquet')]

    for file in files:
        # Ruta completa del archivo
        filePath = os.path.join(folder, file)

        # Filtrar por fecha y guardar en un nuevo archivo Parquet
        df_DateFiltered = dateFilter(filePath)
        df_DateFiltered.to_parquet(filePath)

removeDates(folder)

### 6. RENOMBRAR COLUMNAS

In [ ]:
# Listar todos los archivos Parquet en la carpeta
files = [f for f in os.listdir(folder) if f.endswith('.parquet')]

for file in files:
    # Ruta completa del archivo Parquet
    filePath = os.path.join(folder, file)

    # Leer el archivo Parquet
    df = pd.read_parquet(filePath)

    # Renombrar las columnas
    df = df.rename(columns={"open_time": "date", "open": "price", "quote_asset_volume": "volume_usdt"})

    # Guardar el DataFrame actualizado en el archivo Parquet
    df.to_parquet(filePath, index=False)

### 7. AGRUPAR POR DÍAS

In [ ]:
# Obtener la lista de archivos Parquet en la carpeta
files = [f for f in os.listdir(folder) if f.endswith('.parquet')]

# Iterar sobre cada archivo Parquet
for file in files:
    # Ruta completa del archivo Parquet
    filePath = os.path.join(folder, file)

    # Leer el archivo Parquet
    df = pd.read_parquet(filePath)

    # Convertir la columna "date" al formato de fecha deseado (YYYY-MM-DD)
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Agrupar por día y calcular la media de los valores "price" y la suma de los valores "volume"
    df_GroupedDates = df.groupby('date').agg({'price': 'mean', 'volume_usdt': 'sum'}).reset_index()

    # Guardar el DataFrame resultante en un nuevo archivo Parquet (sobreescribiendo el archivo original)
    df_GroupedDates.to_parquet(filePath, index=False)

### 8. COMPROBAR LOS VALORES NULOS Y 0s

In [ ]:
# Inicializar variables para contar las filas con valores nulos o 0 en la columna "price"
nullTotalCount = 0
ceroTotalCount = 0

# Obtener la lista de archivos Parquet en la carpeta
files = [f for f in os.listdir(folder) if f.endswith('.parquet')]

# Iterar sobre cada archivo Parquet
for file in files:
    # Ruta completa del archivo Parquet
    ruta_parquet = os.path.join(folder, file)

    # Leer el archivo Parquet
    df = pd.read_parquet(ruta_parquet)

    # Contar filas con valores nulos o 0 en la columna "price" de cada archivo
    nullCount = df['price'].isnull().sum()
    ceroCount = (df['price'] == 0).sum()

    # Sumar al total
    nullTotalCount += nullCount
    ceroTotalCount += ceroCount

# Mostrar el total de filas con valores nulos o 0 en la columna "price" en todos los archivos
print("Total de filas con valores nulos en la columna 'price' en todos los archivos:", nullTotalCount)
print("Total de filas con valores 0 en la columna 'price' en todos los archivos:", ceroTotalCount)

### 9. ELECCIÓN DE INTERVALO TEMPORAL

In [ ]:
# Fechas a comprobar (reemplaza con las fechas que desees)
startDate = datetime(2021, 9, 13)
finishDate = datetime(2022, 9, 13)  

def generateRanking(folder):
    # Inicializar un diccionario para almacenar el número de archivos con cada intervalo de 365 días
    intervals = {}
    
    # Iterar sobre los archivos en el directorio
    for archivo in os.listdir(folder):
        if archivo.endswith(".parquet"):
            filePath = os.path.join(folder, archivo)
            
            # Leer el archivo Parquet
            data = pd.read_parquet(filePath)
            
            # Convertir el campo 'date' a objetos de datetime
            data['date'] = pd.to_datetime(data['date'])
            
            # Obtener la fecha mínima y máxima del archivo
            minDate = data['date'].min()
            maxDate = data['date'].max()
            
            # Verificar si ambas fechas son válidas
            if pd.notnull(minDate) and pd.notnull(maxDate):
                # Iterar sobre los intervalos de 365 días dentro del rango de fechas del archivo
                for startDate in pd.date_range(start=minDate, end=maxDate - timedelta(days=365)):
                    finishDate = startDate + timedelta(days=365)
                    
                    # Verificar si ambos extremos del intervalo están presentes en el archivo
                    if (data['date'] >= startDate).any() and (data['date'] <= finishDate).any():
                        # Incrementar el contador para este intervalo
                        intervals[(startDate, finishDate)] = intervals.get((startDate, finishDate), 0) + 1
    
    # Ordenar los intervalos por cantidad de archivos Parquet en orden descendente
    ranking = sorted(intervals.items(), key=lambda x: x[1], reverse=True)
    
    return ranking

# Generar ranking de intervalos de fechas (top 10)
print("\nRanking de intervalos de fechas (Top 10):")
ranking = generateRanking(folder)
for i, ((inicio, fin), n) in enumerate(ranking[:20], 1):
    print(f"{i}. Intervalo: {inicio.strftime('%Y/%m/%d')} - {fin.strftime('%Y/%m/%d')}, Número de criptomonedas: {n}")


### 10. ELIMINAR CRIPTOMONEDAS QUE NO CONTIENEN EL INTERVALO DE FECHAS

In [ ]:
# Iterar sobre los archivos a eliminar y borrarlos
for file in missing_files:
    filePath = os.path.join(folder, file)
    if os.path.exists(filePath):
        os.remove(filePath)
        print(f"Archivo {file} eliminado correctamente.")
    else:
        print(f"El archivo {file} no existe en el directorio.")


### 11. AGREGAR COLUMNAS EMA

In [ ]:
# Obtener la lista de archivos .parquet en la carpeta
files = [file for file in os.listdir(data_folder) if file.endswith('.parquet')]

# Iterar sobre cada archivo .parquet
for file in files:
    # Ruta completa al archivo .parquet
    filePath = os.path.join(folder, file)
    
    # Leer el archivo .parquet en un DataFrame de Pandas
    df = pd.read_parquet(filePath)
    
    # Calcular la EMA50 de la columna 'price'
    df['ema50'] = df['price'].ewm(span=50, adjust=False).mean()
    df['ema100'] = df['price'].ewm(span=100, adjust=False).mean()
    df['ema200'] = df['price'].ewm(span=200, adjust=False).mean()
    
    
    # Escribir el DataFrame modificado de vuelta al archivo .parquet
    df.to_parquet(filePath, index=False)


### 12. NORMALIZAR PRECIOS

In [ ]:
# Obtener la lista de archivos .parquet en la carpeta
files = [file for file in os.listdir(folder) if file.endswith('.parquet')]

# Iterar sobre cada archivo .parquet
for file in files:
    # Ruta completa al archivo .parquet
    filePath = os.path.join(folder, file)
    
    # Leer el archivo .parquet en un DataFrame de Pandas
    df = pd.read_parquet(filePath)
    
    # Calcular el valor máximo de las columnas
    maxPrice = df['price'].max()
    maxEma50 = df['ema50'].max()
    maxEma100 = df['ema100'].max()
    maxEma200 = df['ema200'].max()
    
    # Agregar una nueva columna 'price_norm' con los valores normalizados
    df['price'] = df['price'] / maxPrice
    df['ema50'] = df['ema50'] / maxEma50
    df['ema100'] = df['ema100'] / maxEma100
    df['ema200'] = df['ema200'] / maxEma200
    
    # Escribir el DataFrame modificado de vuelta al archivo .parquet
    df.to_parquet(filePath, index=False)


### 13. ELIMINAR FECHAS FUERA DEL INTERVALO

In [ ]:
def dateFilter(file):
    # Leer el archivo Parquet
    df = pd.read_parquet(file)

    # Reemplaza 'open_time' con el nombre correcto de la columna que contiene la fecha y hora de apertura

    # Convertir la columna 'open_time' a tipo datetime
    df['date'] = pd.to_datetime(df['date'])

    # Filtrar las filas por fecha
    df_DateFiltered = df[(df['date'] >= '2021-09-13') & (df['date'] <= '2022-09-13')]

    return df_DateFiltered

def removeDates(folder):
    # Listar todos los archivos Parquet en la carpeta
    files = [f for f in os.listdir(folder) if f.endswith('.parquet')]

    for file in files:
        # Ruta completa del archivo
        filePath = os.path.join(folder, file)

        # Filtrar por fecha y guardar en un nuevo archivo Parquet
        df_DateFiltered = dateFilter(filePath)
        df_DateFiltered = df_DateFiltered.reset_index()
        df_DateFiltered.to_parquet(filePath)
        

removeDates(folder)

In [ ]:
# Columnas a eliminar
fields = ['index']

def removeFields(folder, fields):
    # Listar todos los archivos Parquet en la carpeta
    files = [f for f in os.listdir(folder) if f.endswith('.parquet')]

    for file in files:
        # Ruta completa
        filePath = os.path.join(folder, file)

        # Leer el archivo Parquet
        df = pd.read_parquet(filePath)

        # Eliminar las columnas especificadas
        df = df.drop(columns=fields, errors='ignore')
        
        # Guardar el nuevo DataFrame sin las columnas en un nuevo archivo Parquet
        df.to_parquet(filePath)


# Procesar archivos Parquet
removeFields(folder, fields)

### TRANSFORMACIÓN DE ARCHIVOS DE MÉTRICAS

In [ ]:
def cluster_distance(cluster1, cluster2):
    # Calcula la distancia entre dos clusters utilizando la distancia mínima entre pares de puntos
    min_distance = float('inf')
    for i in range(len(cluster1)):
        for j in range(len(cluster2)):
            distance = np.linalg.norm(cluster1[i] - cluster2[j])
            if distance < min_distance:
                min_distance = distance
    return min_distance

def intra_cluster_dispersion(cluster):
    # Calcula la dispersión intra-cluster como la media de las distancias entre pares de puntos en el cluster
    total_distance = 0
    num_points = len(cluster)
    for i in range(num_points):
        for j in range(i+1, num_points):
            total_distance += np.linalg.norm(cluster[i] - cluster[j])
    if num_points <= 1:
        return 0
    else:
        return total_distance / (num_points * (num_points - 1) / 2)

def dunn_index(clusters):
    # Calcula el índice de Dunn para un conjunto de clusters
    min_inter_cluster_distance = float('inf')
    max_intra_cluster_dispersion = 0
    num_clusters = len(clusters)
    
    # Calcula la distancia mínima entre clusters
    for i in range(num_clusters):
        for j in range(i+1, num_clusters):
            distance = cluster_distance(clusters[i], clusters[j])
            if distance < min_inter_cluster_distance:
                min_inter_cluster_distance = distance
    
    # Calcula la máxima dispersión intra-cluster
    for i in range(num_clusters):
        dispersion = intra_cluster_dispersion(clusters[i])
        if dispersion > max_intra_cluster_dispersion:
            max_intra_cluster_dispersion = dispersion
    
    # El índice de Dunn es la relación entre la distancia mínima entre clusters y la máxima dispersión intra-cluster
    if max_intra_cluster_dispersion == 0:
        return float('inf')
    else:
        return min_inter_cluster_distance / max_intra_cluster_dispersion


In [ ]:
def equity_score(distribution, total_elements):
    max_elements = max(distribution)
    min_elements = min(distribution)
    if max_elements == min_elements:
        return 1.0
    return min_elements / max_elements

def generate_distributions(total_elements, num_groups):
    return set(itertools.combinations_with_replacement(range(total_elements), num_groups))

def calculate_scores(total_elements, group_numbers):
    rows = []
    for num_groups in group_numbers:
        distributions = generate_distributions(total_elements, num_groups)
        num_distributions = len(distributions)
        for distribution in distributions:
            if sum(distribution) == total_elements:
                score = equity_score(np.bincount(distribution), total_elements)
                std_dev = np.std(distribution)
                rows.append({'groups': num_groups, 'distribution': distribution, 'score': score, 'std_dev': std_dev})
    df = pd.DataFrame(rows)
    df['distribution'] = df['distribution'].apply(lambda x: ';'.join(map(str, x)))
    df = df.groupby('groups').apply(lambda x: x.sort_values(by='std_dev')).reset_index(drop=True)
    return df

# Ejemplo de uso
total_cryptos = 200
group_numbers = [3, 4, 5]
results = calculate_scores(total_cryptos, group_numbers)
results.to_csv('equity_scores.csv', sep=';', index=False)


In [ ]:
# Ruta de la carpeta donde se encuentran los archivos CSV
carpeta_metrics = 'metrics'

# Lista de archivos CSV en la carpeta
archivos_csv = [archivo for archivo in os.listdir(carpeta_metrics) if archivo.endswith('.csv')]

# Procesar cada archivo CSV
for archivo_csv in archivos_csv:
    # Leer el archivo CSV
    ruta_archivo = os.path.join(carpeta_metrics, archivo_csv)
    df = pd.read_csv(ruta_archivo)
    
    # Filtrar las filas que contienen 'media' en la columna 'iteration'
    # y mantener la primera fila (asumida como encabezado)
    filtro = df['iteration'].str.contains('Media', case=False, na=False)
    df_filtrado = df[filtro]
    
    # Guardar el DataFrame filtrado de vuelta al archivo CSV
    df_filtrado.to_csv(ruta_archivo, index=False)

    print(f"Archivo procesado: {ruta_archivo}")


In [ ]:
# Ruta de la carpeta donde se crearán los archivos CSV
carpeta_charts = 'metrics/charts'

# Crear la carpeta si no existe
os.makedirs(carpeta_charts, exist_ok=True)

# Lista de nombres de archivos CSV a crear
archivos_csv = [
    "ema50-silhouette.csv", "ema100-silhouette.csv", "ema200-silhouette.csv", "price-silhouette.csv",
    "ema50-dunn.csv", "ema100-dunn.csv", "ema200-dunn.csv", "price-dunn.csv",
    "ema50-davies.csv", "ema100-davies.csv", "ema200-davies.csv", "price-davies.csv",
    "ema50-calinski.csv", "ema100-calinski.csv", "ema200-calinski.csv", "price-calinski.csv",
    "sk-timeserieskmeans-silhouette.csv", "sk-timeserieskmeans-dunn.csv", "sk-timeserieskmeans-davies.csv", "sk-timeserieskmeans-calinski.csv",
    "sk-timeserieskmedoids-silhouette.csv", "sk-timeserieskmedoids-dunn.csv", "sk-timeserieskmedoids-davies.csv", "sk-timeserieskmedoids-calinski.csv",
    "ts-timeserieskmeans-silhouette.csv", "ts-timeserieskmeans-dunn.csv", "ts-timeserieskmeans-davies.csv", "ts-timeserieskmeans-calinski.csv",
    "ts-kshape-silhouette.csv", "ts-kshape-dunn.csv", "ts-kshape-davies.csv", "ts-kshape-calinski.csv",
    "ts-kernelkmeans-silhouette.csv", "ts-kernelkmeans-dunn.csv", "ts-kernelkmeans-davies.csv", "ts-kernelkmeans-calinski.csv"
]

# Crear cada archivo CSV vacío en la carpeta especificada
for archivo_csv in archivos_csv:
    ruta_archivo = os.path.join(carpeta_charts, archivo_csv)
    # Abrir el archivo en modo de escritura y cerrarlo inmediatamente
    open(ruta_archivo, 'w').close()
    print(f"Archivo creado: {ruta_archivo}")


In [ ]:
# Ruta de la carpeta de entrada (metrics) y la carpeta de salida (charts)
carpeta_metrics = 'metrics'
carpeta_charts = os.path.join(carpeta_metrics, 'charts')

# Crear la carpeta charts si no existe
os.makedirs(carpeta_charts, exist_ok=True)

# Lista de archivos en la carpeta metrics
archivos_metrics = [archivo for archivo in os.listdir(carpeta_metrics) if archivo.endswith('.csv')]

# Lista de archivos en la carpeta charts
archivos_charts = [archivo for archivo in os.listdir(carpeta_charts) if archivo.endswith('.csv')]

# Diccionario de condiciones de coincidencia
condiciones = {
    "sktime-TimeSeriesKMeans": "sk-timeserieskmeans",
    "TimeSeriesKMedoids": "timeserieskmedoids",
    "tslearn-TimeSeriesKMeans": "ts-timeserieskmeans",
    "KShape": "kshape",
    "KernelKMeans": "kernelkmeans"
}

# Función para verificar si un archivo está vacío
def archivo_vacio(ruta):
    return os.path.exists(ruta) and os.stat(ruta).st_size == 0

# Procesar cada archivo CSV en metrics
for archivo_metric in archivos_metrics:
    ruta_archivo_metric = os.path.join(carpeta_metrics, archivo_metric)
    
    # Leer el archivo CSV si no está vacío
    if not archivo_vacio(ruta_archivo_metric):
        df_metric = pd.read_csv(ruta_archivo_metric)
        
        # Determinar los archivos de charts correspondientes
        for key, value in condiciones.items():
            if key in archivo_metric:
                for archivo_chart in archivos_charts:
                    if value in archivo_chart:
                        ruta_archivo_chart = os.path.join(carpeta_charts, archivo_chart)
                        
                        # Leer o inicializar el DataFrame del archivo chart si no está vacío
                        if not archivo_vacio(ruta_archivo_chart):
                            df_chart = pd.read_csv(ruta_archivo_chart)
                        else:
                            df_chart = pd.DataFrame()
                        
                        # Concatenar el DataFrame del archivo metric con el correspondiente DataFrame del archivo chart
                        df_chart = pd.concat([df_chart, df_metric], ignore_index=True)
                        
                        # Guardar el DataFrame actualizado en el archivo chart
                        df_chart.to_csv(ruta_archivo_chart, index=False)
                        print(f"Archivo actualizado: {ruta_archivo_chart}")
    else:
        print(f"El archivo {ruta_archivo_metric} está vacío y no se procesará.")


In [ ]:
# Ruta de la carpeta de entrada (metrics/charts)
carpeta_charts = os.path.join('metrics', 'charts')

# Lista de prefijos de archivos a procesar
prefijos = ['ema50', 'ema100', 'ema200', 'price']

# Ruta del archivo ema100-silhouette.csv
archivo_ema100_silhouette = os.path.join(carpeta_charts, 'ema100-silhouette.csv')

# Leer el archivo ema100-silhouette.csv para obtener la columna 'method'
df_ema100_silhouette = pd.read_csv(archivo_ema100_silhouette)

# Comprobar si la columna 'method' existe en el DataFrame
if 'method' in df_ema100_silhouette.columns:
    method_values = df_ema100_silhouette['method']
else:
    raise ValueError("La columna 'method' no existe en el archivo ema100-silhouette.csv")

# Lista de archivos en la carpeta charts
archivos_charts = [archivo for archivo in os.listdir(carpeta_charts) if archivo.endswith('.csv')]

# Función para verificar si el nombre del archivo empieza con alguno de los prefijos dados
def empieza_con_prefijo(nombre_archivo, prefijos):
    return any(nombre_archivo.startswith(prefijo) for prefijo in prefijos)

# Procesar cada archivo CSV en charts
for archivo_chart in archivos_charts:
    if empieza_con_prefijo(archivo_chart, prefijos):
        ruta_archivo_chart = os.path.join(carpeta_charts, archivo_chart)
        
        # Leer el archivo CSV
        df_chart = pd.read_csv(ruta_archivo_chart)
        
        # Comprobar si la columna 'iteration' existe en el DataFrame
        if 'iteration' in df_chart.columns:
            # Sustituir la columna 'iteration' con los valores de la columna 'method'
            df_chart['iteration'] = method_values
            
            # Guardar el DataFrame actualizado en el archivo chart
            df_chart.to_csv(ruta_archivo_chart, index=False)
            print(f"Columna 'iteration' sustituida en: {ruta_archivo_chart}")
        else:
            print(f"La columna 'iteration' no existe en: {ruta_archivo_chart}")


In [ ]:
# Ruta de la carpeta de entrada (metrics/charts)
carpeta_charts = os.path.join('metrics', 'charts')

# Lista de archivos en la carpeta charts
archivos_charts = [archivo for archivo in os.listdir(carpeta_charts) if archivo.endswith('.csv')]

# Diccionario de condiciones y las columnas a eliminar
condiciones_columnas = {
    "calinski": ["Silhouette", "Dunn", "Davies-Bouldin", "time"],
    "davies": ["Silhouette", "Dunn", "Calinski-Harabasz", "time"],
    "dunn": ["Silhouette", "Calinski-Harabasz", "Davies-Bouldin", "time"],
    "time": ["Silhouette", "Dunn", "Davies-Bouldin", "Calinski-Harabasz"],
    "silhouette": ["Calinski-Harabasz", "Dunn", "Davies-Bouldin", "time"]
}

# Procesar cada archivo CSV en charts
for archivo_chart in archivos_charts:
    ruta_archivo_chart = os.path.join(carpeta_charts, archivo_chart)
    
    # Leer el archivo CSV
    df_chart = pd.read_csv(ruta_archivo_chart)
    
    # Determinar las columnas a eliminar según el nombre del archivo
    columnas_a_eliminar = []
    for key, columnas in condiciones_columnas.items():
        if key in archivo_chart:
            columnas_a_eliminar.extend(columnas)
    
    # Eliminar las columnas especificadas si existen en el DataFrame
    columnas_a_eliminar = list(set(columnas_a_eliminar))  # Eliminar duplicados
    columnas_existentes_a_eliminar = [col for col in columnas_a_eliminar if col in df_chart.columns]
    
    if columnas_existentes_a_eliminar:
        df_chart.drop(columns=columnas_existentes_a_eliminar, inplace=True)
        # Guardar el DataFrame actualizado en el archivo chart
        df_chart.to_csv(ruta_archivo_chart, index=False)
        print(f"Columnas {columnas_existentes_a_eliminar} eliminadas en: {ruta_archivo_chart}")
    else:
        print(f"No hay columnas para eliminar en: {ruta_archivo_chart}")


In [ ]:
# Ruta de la carpeta de entrada (metrics/charts)
carpeta_charts = os.path.join('metrics', 'charts')

# Lista de archivos en la carpeta charts
archivos_charts = [archivo for archivo in os.listdir(carpeta_charts) if archivo.endswith('.csv')]

# Procesar cada archivo CSV en charts
for archivo_chart in archivos_charts:
    ruta_archivo_chart = os.path.join(carpeta_charts, archivo_chart)
    
    # Leer el archivo CSV
    df_chart = pd.read_csv(ruta_archivo_chart)
    
    # Verificar si la columna 'iteration' existe y renombrarla a 'method'
    if 'iteration' in df_chart.columns:
        df_chart.rename(columns={'iteration': 'method'}, inplace=True)
        
        # Guardar el DataFrame actualizado en el archivo chart
        df_chart.to_csv(ruta_archivo_chart, index=False)
        print(f"Columna 'iteration' renombrada a 'method' en: {ruta_archivo_chart}")
    else:
        print(f"La columna 'iteration' no existe en: {ruta_archivo_chart}")


In [ ]:
import os
import pandas as pd

# Ruta de la carpeta de entrada (metrics/charts)
carpeta_charts = os.path.join('metrics', 'charts')

# Lista de archivos en la carpeta charts
archivos_charts = [archivo for archivo in os.listdir(carpeta_charts) if archivo.endswith('.csv')]

# Columnas a conservar
columnas_a_conservar = ['clusters', 'iteration', 'time']

# Procesar cada archivo CSV en charts
for archivo_chart in archivos_charts:
    if archivo_chart.endswith('time.csv'):
        ruta_archivo_chart = os.path.join(carpeta_charts, archivo_chart)
        
        # Leer el archivo CSV
        df_chart = pd.read_csv(ruta_archivo_chart)
        
        # Conservar solo las columnas especificadas
        columnas_existentes_a_conservar = [col for col in columnas_a_conservar if col in df_chart.columns]
        df_chart = df_chart[columnas_existentes_a_conservar]
        
        # Guardar el DataFrame actualizado en el archivo chart
        df_chart.to_csv(ruta_archivo_chart, index=False)
        print(f"Archivo procesado y actualizado: {ruta_archivo_chart}")


In [ ]:
# Ruta del archivo sk-timeserieskmeans-silhouette.csv
ruta_archivo_base = os.path.join('metrics', 'charts', 'sk-timeserieskmeans-silhouette.csv')

# Leer el archivo base para obtener la columna 'field'
df_base = pd.read_csv(ruta_archivo_base)

# Verificar si la columna 'field' existe en el DataFrame base
if 'field' in df_base.columns:
    columna_field = df_base['field']
else:
    raise ValueError("La columna 'field' no existe en el archivo base.")

# Ruta de la carpeta de entrada (metrics/charts)
carpeta_charts = os.path.join('metrics', 'charts')

# Lista de archivos en la carpeta charts
archivos_charts = [archivo for archivo in os.listdir(carpeta_charts) if archivo.endswith('.csv')]

# Procesar cada archivo CSV en charts
for archivo_chart in archivos_charts:
    ruta_archivo_chart = os.path.join(carpeta_charts, archivo_chart)
    
    # Leer el archivo CSV
    df_chart = pd.read_csv(ruta_archivo_chart)
    
    # Verificar si la columna 'iteration' existe en el DataFrame
    if 'iteration' in df_chart.columns:
        # Sustituir la columna 'iteration' con la columna 'field' del archivo base
        df_chart['iteration'] = columna_field
        
        # Guardar el DataFrame actualizado en el archivo chart
        df_chart.to_csv(ruta_archivo_chart, index=False)
        print(f"Columna 'iteration' sustituida por 'field' en: {ruta_archivo_chart}")
    else:
        print(f"La columna 'iteration' no existe en: {ruta_archivo_chart}")


In [ ]:
# Ruta de la carpeta de entrada (metrics)
carpeta_metrics = 'metrics/charts-fields'

# Lista para almacenar los archivos CSV encontrados
archivos_csv = []

# Recorrer los directorios y archivos dentro de la carpeta metrics
for root, dirs, files in os.walk(carpeta_metrics):
    for file in files:
        if file.endswith('.csv'):
            archivos_csv.append(os.path.join(root, file))

# Procesar cada archivo CSV encontrado
for archivo in archivos_csv:
    # Leer el archivo CSV
    df = pd.read_csv(archivo)
    
    # Verificar si la columna 'iteration' existe en el DataFrame
    if 'iteration' in df.columns:
        # Cambiar el nombre de la columna 'iteration' a 'field'
        df.rename(columns={'iteration': 'field'}, inplace=True)
        
        # Guardar el DataFrame actualizado en el archivo CSV
        df.to_csv(archivo, index=False)
        print(f"Columna 'iteration' renombrada a 'field' en: {archivo}")
    else:
        print(f"La columna 'iteration' no existe en: {archivo}")
